In [5]:
import requests

url = 'http://localhost:8081/sentiment'

# 예제 JSON 데이터
data = {
    "data":"오늘 배가 많이 고파서 아무것도 못먹었다"
}

# JSON 데이터를 포함한 POST 요청
response = requests.post(url, json=data, headers={'Content-Type': 'application/json'})

print(response.status_code)  # HTTP 상태 코드 출력  
print(response.json())       # JSON 응답 본문 출력


200
{'sentiment analysis': [{'label': 'NEGATIVE', 'score': 0.98365}]}


In [18]:
import boto3
import botocore
import tarfile
import os

from dotenv import load_dotenv
import os

load_dotenv()
aws_access_key_id = os.getenv('aws_access_key_id')
aws_secret_access_key =  os.getenv('aws_secret_access_key')
region_name =  os.getenv('region_name')

s3 = boto3.client('s3',
            aws_access_key_id = aws_access_key_id,
            aws_secret_access_key = "RKuYAPErS9zbLrNZckBWBrHZ4hUTPHOhmO5/fa/7",
            region_name=region_name
        ) 

# S3 버킷 이름과 파일 키 설정


bucket_name = os.getenv('bucket_name')
file_key =  os.getenv('file_key')
local_file_path =  os.getenv('local_file_path')  # 로컬에 저장할 파일 경로

# 모델 파일을 다운로드할 로컬 경로 설정
local_model_path = 'model.tar.gz'

# 모델 디렉토리 생성
model_dir = 'model'
os.makedirs(model_dir, exist_ok=True)

# # S3에서 모델 파일 다운로드
try:
    s3.head_object(Bucket=bucket_name, Key=file_key)
    s3.download_file(bucket_name, file_key, local_file_path)
    print(f'Downloaded {model_key} to {local_model_path}')
except Exception as e:
    print(f"Error downloading file: {e}")

# # 모델 파일 추출
try:
    with tarfile.open(local_model_path, 'r:gz') as tar:
        tar.extractall(model_dir)
        print(f'Extracted {local_model_path} to {model_dir}')
except Exception as e:
    print(f"Error extracting tar file: {e}")

# # 추출된 파일 확인
extracted_files = []
for root, dirs, files in os.walk(model_dir):
    for file in files:
        extracted_files.append(os.path.join(root, file))

print(f'Extracted files: {extracted_files}')

Downloaded model-output/tensorflow-training-2024-06-12-01-47-16-809/output/model.tar.gz to model.tar.gz
Extracted model.tar.gz to model
Extracted files: ['model/keras_metadata.pb', 'model/saved_model.pb', 'model/fingerprint.pb', 'model/variables/variables.index', 'model/variables/variables.data-00000-of-00001']


In [19]:
import boto3
import tarfile
import os
import tensorflow as tf
import numpy as np
from keras.layers import TFSMLayer



# # 추출된 모델 디렉토리 경로 설정 (saved_model.pb 파일의 경로)
saved_model_dir = 'model'
model_save_path = 'tf_model'

try:
    model_layer = TFSMLayer(saved_model_dir, call_endpoint="serving_default")
    print(f'Model loaded from {saved_model_dir}')
except Exception as e:
    print(f"Error loading model with TFSMLayer: {e}")

Model loaded from model


In [7]:
input_data = np.random.rand(1, 28, 28).astype(np.float32) # 1개 행에 28개 들어있고 거기에 또 28개 들어있음
input_tensor = tf.constant(np.expand_dims(input_data, axis=-1))  # 차원 추가: (1, 28, 28, 1)

# 모델 예측
try:
    predictions = model_layer(input_tensor)
except Exception as e:
    print(f"Error making predictions: {e}")

In [4]:
# print(input_data)

In [8]:
input_data = np.random.rand(1, 28, 28).astype(np.float32) # 1개 행에 28개 들어있고 거기에 또 28개 들어있음
input_tensor = tf.constant(np.expand_dims(input_data, axis=-1))  # 차원 추가: (1, 28, 28, 1)
predictions = model_layer(input_tensor)
print(input_data)

[[[0.8955105  0.1016972  0.8729508  0.8626645  0.7843114  0.07212149
   0.7098252  0.34969658 0.5992268  0.31040907 0.8610637  0.56712747
   0.69998795 0.5155321  0.90513235 0.81940806 0.92453367 0.4103348
   0.15873802 0.92269295 0.3722432  0.5708968  0.8293317  0.9314682
   0.6293876  0.59560573 0.47916484 0.69248337]
  [0.5815337  0.7094619  0.8832541  0.5800571  0.29894146 0.6471991
   0.45902398 0.6706652  0.46243328 0.8412372  0.23173273 0.86089844
   0.50180936 0.06613632 0.929212   0.51505417 0.8577952  0.42481452
   0.95223624 0.5195871  0.30811182 0.19263211 0.6478622  0.83126503
   0.70861846 0.5100398  0.18544692 0.12045297]
  [0.4626717  0.41706052 0.30325267 0.3765657  0.79879534 0.23522457
   0.7974669  0.71672183 0.04791787 0.82577217 0.5420557  0.8583014
   0.44197142 0.24838595 0.13533074 0.7869518  0.14156152 0.3400218
   0.86156523 0.31519797 0.90974855 0.8709362  0.1357537  0.00202303
   0.7118813  0.7563036  0.5533232  0.8116334 ]
  [0.646019   0.24269755 0.711602

In [9]:
import socket

def get_local_ip():
    hostname = socket.gethostname()
    local_ip = socket.gethostbyname(hostname)
    return local_ip

print("Local IP Address:", get_local_ip())

Local IP Address: 172.31.13.91


### VPC 통신 테스트용 모델

In [ ]:
from flask import Flask, request, jsonify
import numpy as np
import tensorflow as tf

app = Flask(__name__)

@app.route('/predict', methods=['POST'])
def predict():
    try:
        data = request.json
        print("입력 데이터:", data)
        
        input_data = np.random.rand(1, 28, 28).astype(np.float32) # 1개 행에 28개 들어있고 거기에 또 28개 들어있음
        input_tensor = tf.constant(np.expand_dims(input_data, axis=-1))  # 차원 추가: (1, 28, 28, 1)
        
        # 모델 예측 (예시로서 model_layer 함수 사용)
        predictions = model_layer(input_tensor)  # model_layer 함수가 TensorFlow 텐서를 반환한다고 가정
        print(predictions['dense_1'].numpy())

        return jsonify({'predicted_class': predictions['dense_1'].numpy().tolist() })

    except Exception as e:
        return jsonify({'error': str(e)})

if __name__ == "__main__":
    from waitress import serve
    serve(app, host="0.0.0.0", port=8080)